In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as cmo
import cartopy.crs as ccrs

sys.path.append("..")

from plotfunctions import *

np.seterr(all='ignore')
import warnings
warnings.filterwarnings("ignore")

%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .15
mpl.rcParams['figure.subplot.top']    = .94
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .99
mpl.rcParams['figure.subplot.wspace'] = .02
mpl.rcParams['figure.subplot.hspace'] = .0

In [ ]:
"""Boundaries Adusumilli"""
X1,X2 = 2120,2480
Y1,Y2 = {},{}
Y1['CrossDots'] = 3360
Y2['CrossDots'] = 3680
Y1['Thwaites_e'] = 3680
Y2['Thwaites_e'] = 4000
Y1['PineIsland']  = 4000
Y2['PineIsland']  = 4240

"""Boundaries MITgcm"""
y1,y2 = -75.45,-74.05
x1,x2 = {},{}
x1['CrossDots'] = 245.3
x2['CrossDots'] = 251
x1['Thwaites_e'] = 251
x2['Thwaites_e'] = 257
x1['PineIsland']  = 257
x2['PineIsland']  = 262

"""Locations for text"""
plon,plat = {},{}
plon['CrossDots'] = 250.5
plat['CrossDots'] = -74.4
plon['Thwaites_e'] = 253.5
plat['Thwaites_e'] = -74.5
plon['PineIsland'] = 257
plat['PineIsland'] = -74.8

In [ ]:
#Parameters MITgcm
timep= slice("1989-1-1","2018-12-31")

#Parameters Layer
#forc = 'holland'
forc = 'tanh_Tdeep0.5_ztcl-550'
#forc = 'tanh_Tdeep0.4_ztcl-550'

#Results date
#date = '2021_02_03/'
date = ''

#Extension for filename
#ext = ''
ext = '_lowvis'

In [ ]:
def printvals(ax,geom,melt):
    ax.text(plon[geom],plat[geom],f'{np.nanmean(melt):.1f}\n ({np.nanstd(melt):.1f})',transform=ccrs.PlateCarree(),c='w',ha='center')

In [ ]:
fig = plt.figure(figsize=(8,7))
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=-13)

"""Draft"""
ax = fig.add_subplot(141,projection=proj)
makebackground(ax)
ds = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(3000,4000),y=slice(6500,9000))
ds = add_lonlat(ds)
draft = xr.where(ds.mask==3,(ds.surface-ds.thickness).astype('float64'),np.nan)
IM = plt.pcolormesh(ds.lon,ds.lat,draft,vmin=-1500,vmax=0,cmap=plt.get_cmap('cmo.deep_r'),transform=ccrs.PlateCarree(),shading='auto')

"""Colorbar"""
axins = inset_axes(ax,width="100%",height="4%",loc='lower left',bbox_to_anchor=(0.,-.08, 1, 1),bbox_transform=ax.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='min',orientation='horizontal')
cbar.set_label('Depth [m]', labelpad=0)

ax.set_title('a) Draft',loc='left')


"""Observations"""
ax = fig.add_subplot(142,projection=proj)
makebackground(ax)
geom = 'PineIsland'
ds = xr.open_dataset('../../data/davidshean/mos-tile-0_init_masked.nc')
#ds = xr.open_dataset('../../data/davidshean/mos-tile-0_masked.nc')
ds = add_lonlat(ds)
melt = ds.Band1
printvals(ax,geom,melt)
IM = plotnormmelt(ax,ds.lon,ds.lat,melt)

geom = 'CrossDots'
ds = xr.open_dataset('../../data/goldberg/grl58468-sup-0002-2018gl080383-data_set_si-s01.nc')
ds['x'] = ds.x_stereo
ds['y'] = ds.y_stereo
ds = add_lonlat(ds)
melt = ds.melt_model2
melt = xr.where(melt==0,np.nan,melt)
printvals(ax,geom,melt)
IM = plotnormmelt(ax,ds.lon,ds.lat,melt)


ax.set_title('b) Observations',loc='left')

#ax1 = fig.add_subplot(142,projection=proj)
#makebackground(ax1)
#ax1.set_title('Adusumilli et al')
ax2 = fig.add_subplot(143,projection=proj)
makebackground(ax2)
ax2.set_title('c) 3D Ocean model',loc='left')
ax3 = fig.add_subplot(144,projection=proj)
makebackground(ax3)
ax3.set_title('d) Layer',loc='left')

for geom in ['PineIsland','Thwaites_e','CrossDots']:
    #"""Adusumilli"""

    #ds = xr.open_dataset('../../data/bb0448974g_3_1.h5')
    #ds = ds.isel(phony_dim_0=slice(Y1[geom],Y2[geom]),phony_dim_1=slice(X1,X2),phony_dim_2=0)
    #ds = add_lonlat(ds)

    #melt = xr.where(np.isnan(ds.w_b),0,ds.w_b)
    #melt += xr.where(np.isnan(ds.w_b_interp),0,ds.w_b_interp)
    #melt = xr.where(melt == 0,np.nan,melt)
    
    #printvals(ax1,geom,melt)
    #IM = plotnormmelt(ax1,ds.lon,ds.lat,melt)

    """MITgcm output"""

    ds = xr.open_dataset('../../data/paulholland/PAS_851/state2D.nc')
    ds = ds.sel(LONGITUDE=slice(x1[geom],x2[geom]),LATITUDE=slice(y1,y2),TIME=timep)
    ds = ds.mean(dim='TIME')
    lon = ds.LONGITUDE
    lat = ds.LATITUDE-.05
    
    melt = -ds.SHIfwFlx * 365.25*86400/920
    melt = xr.where(melt==0,np.nan,melt)
    
    printvals(ax2,geom,melt)
    IM = plotnormmelt(ax2,lon,lat,melt)

    """Layer"""
    ds = xr.open_dataset(f'../../results/{date}Layer_{geom}_{forc}.nc')
    ds = add_lonlat(ds)
    
    melt = np.where(ds.mask==3,ds.melt,np.nan)

    printvals(ax3,geom,melt)
    IM = plotnormmelt(ax3,ds.lon,ds.lat,melt)

"""Colorbar"""
ax = fig.add_subplot(142)
ax.set_visible(False)
axins = inset_axes(ax,width="300%",height="4%",loc='lower left',bbox_to_anchor=(0.,-.08, 1, 1),bbox_transform=ax.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='both',orientation='horizontal')

cbar.set_label('Spatial standardised melt rate anomaly []', labelpad=0)

"""Save figure"""
plt.savefig(f"../../figures/Validation_standardized{ext}.png",dpi=300)
plt.show()